# This is a environment to test, prototype, and develop CCBA Library.

In [1]:
import sys

import numpy as np
import pandas as pd

sys.path.insert(0, '/Users/Kwat/binf/ccba')
import ccba_lib

<12:28:43.267826> Checking dependencies ...
<12:28:43.270066> Using the following packages:
<12:28:43.271697> 	matplotlib v1.5.1
<12:28:43.271714> 	numpy v1.10.4
<12:28:43.271723> 	pandas v0.18.0
<12:28:43.271743> 	rpy2 v2.7.9
<12:28:43.271751> 	scikit-learn v0.17.1
<12:28:43.271758> 	scipy v0.17.0
<12:28:43.271765> 	seaborn v0.7.0


In [2]:
## Make test data
nrow = 10
ncol = 50
ref = np.random.rand(ncol)
df = pd.DataFrame(np.random.random((nrow, ncol)))
df.index = ['GENE {}'.format(i) for i in range(nrow)]
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
GENE 0,0.051271,0.921401,0.191297,0.985135,0.595941,0.311613,0.628693,0.264480,0.930568,0.637047,...,0.275756,0.596258,0.801343,0.520996,0.902098,0.858173,0.073735,0.320174,0.951701,0.056643
GENE 1,0.174270,0.417102,0.968662,0.729003,0.153189,0.649962,0.197777,0.182436,0.584316,0.917793,...,0.190018,0.867556,0.265831,0.291223,0.474530,0.684393,0.141573,0.689306,0.106853,0.441907
GENE 2,0.989138,0.756691,0.207953,0.835845,0.414697,0.392975,0.760158,0.177299,0.964068,0.056773,...,0.711543,0.050046,0.256184,0.380116,0.928233,0.665801,0.865007,0.318722,0.627798,0.911761
GENE 3,0.953946,0.143804,0.177641,0.765037,0.609406,0.651566,0.045911,0.516741,0.890781,0.764801,...,0.272247,0.193629,0.542403,0.073893,0.758281,0.154146,0.798682,0.056344,0.313064,0.669908
GENE 4,0.578882,0.014461,0.473863,0.967510,0.457812,0.189865,0.661590,0.420299,0.197117,0.198698,...,0.792833,0.895478,0.550158,0.909920,0.259950,0.199784,0.828053,0.852697,0.897329,0.048526
GENE 5,0.805414,0.057471,0.027730,0.741032,0.342279,0.014748,0.631572,0.958271,0.049869,0.793396,...,0.328902,0.311373,0.358680,0.880204,0.253496,0.245316,0.702735,0.674238,0.859618,0.111661
GENE 6,0.442554,0.878732,0.873483,0.271223,0.194238,0.513945,0.207285,0.497829,0.626682,0.238229,...,0.959766,0.985660,0.844935,0.886476,0.418662,0.868782,0.454037,0.681551,0.886062,0.080040
GENE 7,0.954337,0.328683,0.609777,0.344893,0.406600,0.219783,0.533871,0.476177,0.568830,0.521045,...,0.377055,0.909171,0.783395,0.565553,0.044490,0.514733,0.265224,0.412123,0.961553,0.362470
GENE 8,0.518900,0.424272,0.958373,0.105200,0.088094,0.441740,0.264603,0.184418,0.907635,0.796598,...,0.406262,0.623524,0.401726,0.516787,0.924307,0.155526,0.064867,0.225622,0.000834,0.326111
GENE 9,0.157120,0.086947,0.970490,0.590287,0.516507,0.176259,0.180591,0.604037,0.731047,0.118600,...,0.203036,0.976647,0.303970,0.682681,0.588631,0.274092,0.584769,0.355322,0.287011,0.787916


In [5]:
nmf_result = ccba_lib.analyze.nmf(df, [3])

<12:28:55.228069> Perfomring NMF with k=3 ...


In [ ]:
# make_heatmap_panel is in the ccba/library/ccba.py
# make_heatmap_panel calls plot_heatmap_panel, which is in the ccba/library/visualize.py
# make_heatmap_panel computes IC of each df's row and the ref (target).
# In the future, we can add other computation within this function so it can compute P-Val and FDR as well.
make_heatmap_panel(df, ref, ['IC', 'P-Val'])

In [ ]:
import cProfile
#import timeit
import time
from library.ccba import *
from scipy.stats import pearsonr
import seaborn as sns
%matplotlib inline

def runtime(function, n_range, verbose=False):
    ns = []
    runtimes = []
    for i in n_range:
        n = 10 * (i + 1)
        if verbose:
            print(n)
        x = np.random.rand(n)
        y = np.random.rand(n)
        t0 = time.time()

        function(x, y)

        runtime = time.time() - t0
        ns.append(n)
        runtimes.append(runtime)
        #cProfile.run(func)
    return ns, runtimes

ns, runtimes = runtime(information_coefficient, n_range=list(range(1000)))
ax = sns.pointplot(x=ns, y=runtimes)

ns, runtimes = runtime(pearsonr, n_range=list(range(1000)), verbose=False)
ax = sns.pointplot(x=ns, y=runtimes)

In [ ]:
# Feature selection test with real data for IC, CMI diff, and CMI ratio association metrics
## Ranking 60 genes + target profile with respect to the activation profile of KRAS component 6 (BRAF/MAPK)

def plot_heatmap_panel_v2(target, df, row_annot, title, t_type="auto", sort_target=True, class_labels=None, cbar=False):
    """
    Plot horizonzal heatmap panels.s
    """     
    ncol = df.shape[1]
    nrow = df.shape[0]
    df2 = pd.DataFrame(np.zeros((nrow, ncol)))
    CMAP_CONTINUOUS =  mpl.cm.bwr 
    CMAP_BINARY = sns.light_palette("black", n_colors=128, as_cmap=True)
    CMAP_CATEGORICAL = mpl.cm.Set2 
    f_width = np.where(ncol/5 < 5, 5, np.where(ncol/5 > 7, 7, ncol/5))
    f_height = np.where(nrow < 25, nrow/1.5, nrow/2.5)
    figure_size = (f_width, f_height)

    # Target pre-processing
    if t_type == "binary" or (t_type == "auto" and len(np.unique(target.iloc[0,:]))) == 2:
        cmap_target = CMAP_BINARY
        (t_vmin, t_vmax) = (0, 1)
    elif t_type == "categ" or (t_type == "auto" and len(np.unique(target.iloc[0,:])) > 2 and 
                               len(np.unique(target.iloc[0,:])) < ncol): 
        cmap_target = CMAP_CATEGORICAL
        (t_vmin, t_vmax) = (0, len(np.unique(target.iloc[0:]) - 1))   
    else:  # t_type = "cont"      
        cmap_target = CMAP_CONTINUOUS
        target.iloc[0,:] = (target.iloc[0,:] - np.mean(target.iloc[0,:]))/np.std(target.iloc[0,:])
        (t_vmin, t_vmax) = (-2.5, 2.5)
        means = df.mean(1, skipna=True)
        stds = df.std(1, skipna=True) 

    # Feature pre-processing
    row_names = df.index 
    max_len_row = max([ len(x) for x in row_names])
    if len(np.unique(df)) == 2:
        cmap_features = CMAP_BINARY
        (f_vmin, f_vmax) = (0, 1)
        df = df + 0.25
    else:
        cmap_features = CMAP_CONTINUOUS
        (f_vmin, f_vmax) = (-2.5, 2.5)
        means = df.mean(1, skipna=True)
        stds = df.std(1, skipna=True)
        for i in range(nrow):
            for j in range(ncol):
                df.values[i, j] = (df.values[i, j] - means[i])/stds[i]
         
    if sort_target == True:
        t_order = list(reversed(np.argsort(target.values[0,:], kind='quicksort')))
        target = target.reindex_axis(target.columns[t_order], axis=1) 
        df2 = df.reindex_axis(df.columns[t_order], axis=1)
  
        # Computer locations for class labels (for binary or categorical)
        if (t_type == "binary" or t_type == "categ") and class_labels != None:
            boundaries = np.zeros(len(np.unique(target.iloc[0,:])))
            locs_labels = np.zeros(len(np.unique(target.iloc[0,:])))
            k = 0
            for i in range(1, ncol):
                if target.iloc[0, i] != target.iloc[0, i-1]:
                    boundaries[k] = i 
                    k = k + 1
            boundaries[len(boundaries) - 1] = ncol
            locs_labels[0] = boundaries[0]/2
            for k in range(1, len(locs_labels)):
                locs_labels[k] = boundaries[k] - (boundaries[k] - boundaries[k - 1])/2.0  
        
    ## Initialize figure
    fig = plt.figure(figsize=figure_size)
    
    ax1 = plt.subplot2grid((nrow + 1, 1), (0, 0))

    sns.heatmap(target, vmin=t_vmin, vmax=t_vmax, robust=True, center=None, mask=None,
            square=False, cmap=cmap_target, linewidth=0.0, linecolor='b',
            annot=False, fmt=None, annot_kws={}, xticklabels=False, 
            yticklabels=['  '], cbar=False)
    ax1.text(-0.1, 0.2, target.index[0], fontsize=13, horizontalalignment='right', fontweight='bold')
    ax1.text(ncol/2, 1.6, title, fontsize=16, horizontalalignment='center', fontweight='bold')
    if (t_type == "binary" or t_type == "categ") and class_labels != None:
        for k in range(len(locs_labels)):
            ax1.text(locs_labels[k], 0.25, class_labels[k], fontsize=13, horizontalalignment='center', fontweight='bold')
   
    ax2 = plt.subplot2grid((nrow + 1, 1), (0, 1), rowspan=nrow)
    sns.heatmap(df2, vmin=f_vmin, vmax=f_vmax, robust=True, center=None, mask=None,
            square=False, cmap=cmap_features, linewidth=0.0, linecolor='b',
            annot=False, fmt=None, annot_kws={}, xticklabels=False,                
            yticklabels=[' ' for i in range(nrow)], cbar=cbar, cbar_kws={"orientation": "horizontal"})  
    for i in range(nrow):
        ax2.text(-0.1, nrow - i - 1 + 0.3, row_names[i], fontsize=13, horizontalalignment='right', fontweight='bold')
        ax2.text(ncol + 0.1, nrow - i - 1 + 0.3, row_annot[i], fontsize=13, fontweight='bold')
    ax2.text(ncol + 1, nrow + 0.3, row_annot.name, fontsize=13, fontweight='bold')        
    
    fig.tight_layout()    
    plt.show(fig) 
    
    return

## Feature selection using Information Coefficient

start_time = time.time()

gct1 = read_gct('/Users/tamayo/UCSD_2015/signatures/paper2/analysis/top_hits.gct', fill_na=0)
gct2 = read_gct('/Users/tamayo/UCSD_2015/signatures/Onco-GPS/CCLE_RNAseq_KRAS_SALE_Late.No_HAEM.k_9.H.v1.gct', fill_na=0)

common_cols = list(set(gct1.columns) & set(gct2.columns))
A = gct1[common_cols]
B = gct2[common_cols]
target = pd.DataFrame(B.loc['KRAS_SALE_Late_Comp_C6_9'])
target = target.T

IC = np.zeros(A.shape[0])
for i in range(A.shape[0]):
    IC[i] = round(compute_information_coefficient(x=target.iloc[0,:], y=A.iloc[i,:]), 3)  
                
IC_order = list(np.argsort(IC, kind='quicksort'))
IC = IC[IC_order]
A_sorted = A.reindex(A.index[IC_order]) 

end_time = time.time() 
print("Feature selection time %s seconds " % (end_time - start_time))

IC_val = pd.Series(IC[:A.shape[0]], name = "IC")
features = A_sorted.head(A.shape[0])
plot_heatmap_panel_v2(target, features, IC_val, "KRAS component 6 (BRAF/MAPK) co-essential genes")

## Feature selection using CMI diff

start_time = time.time()

gct1 = read_gct('/Users/tamayo/UCSD_2015/signatures/paper2/analysis/top_hits.gct', fill_na=0)
gct2 = read_gct('/Users/tamayo/UCSD_2015/signatures/Onco-GPS/CCLE_RNAseq_KRAS_SALE_Late.No_HAEM.k_9.H.v1.gct', fill_na=0)

common_cols = list(set(gct1.columns) & set(gct2.columns))
A = gct1[common_cols]
B = gct2[common_cols]
target = pd.DataFrame(B.loc['KRAS_SALE_Late_Comp_C6_9'])
target = target.T

CMI_diff_vals = np.zeros(A.shape[0])
for i in range(A.shape[0]):
    CMI_diff_vals[i] = round(CMI_diff(x=target.iloc[0,:], y=A.iloc[i,:]), 3)  
                
CMI_order = list(np.argsort(CMI_diff_vals, kind='quicksort'))
CMI_diff_vals = CMI_diff_vals[CMI_order]
A_sorted = A.reindex(A.index[CMI_order]) 

end_time = time.time() 
print("Feature selection time %s seconds " % (end_time - start_time))

CMI_val = pd.Series(CMI_diff_vals[:A.shape[0]], name = "CMI diff")
features = A_sorted.head(A.shape[0])
plot_heatmap_panel_v2(target, features, CMI_val, "KRAS component 6 (BRAF/MAPK) co-essential genes")

## Feature selection using CMI ratio

start_time = time.time()


gct1 = read_gct('/Users/tamayo/UCSD_2015/signatures/paper2/analysis/top_hits.gct', fill_na=0)
gct2 = read_gct('/Users/tamayo/UCSD_2015/signatures/Onco-GPS/CCLE_RNAseq_KRAS_SALE_Late.No_HAEM.k_9.H.v1.gct', fill_na=0)

common_cols = list(set(gct1.columns) & set(gct2.columns))
A = gct1[common_cols]
B = gct2[common_cols]
target = pd.DataFrame(B.loc['KRAS_SALE_Late_Comp_C6_9'])
target = target.T

CMI_ratio_vals = np.zeros(A.shape[0])
for i in range(A.shape[0]):
    CMI_ratio_vals[i] = round(CMI_ratio(x=target.iloc[0,:], y=A.iloc[i,:]), 3)  
                
CMI_order = list(np.argsort(CMI_ratio_vals, kind='quicksort'))
CMI_ratio_vals = CMI_ratio_vals[CMI_order]
A_sorted = A.reindex(A.index[CMI_order]) 

end_time = time.time() 
print("Feature selection time %s seconds " % (end_time - start_time))

CMI_val = pd.Series(CMI_ratio_vals[:A.shape[0]], name = "CMI ratio")
features = A_sorted.head(A.shape[0])

plot_heatmap_panel_v2(target, features, CMI_val, "KRAS component 6 (BRAF/MAPK) co-essential genes")